Part-2 
https://dl.acm.org/doi/epdf/10.1145/3704137.3704173

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from medmnist import PathMNIST

# Define dataset name and parameters
dataset_name = "pathmnist"  # Change to your dataset of choice
batch_size = 32
num_epochs = 50
learning_rate = 0.001
patience = 10
num_classes = 9  # Adjust based on the dataset

# Load dataset
dataset = PathMNIST(split="train", download=True)

# Preprocessing function
def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = (image - 0.5) / 0.5  # Normalize to [-1, 1]
    return image, tf.one_hot(label, depth=num_classes)

# Convert dataset to TensorFlow format
images = dataset.imgs
labels = dataset.labels.squeeze()

dataset_tf = tf.data.Dataset.from_tensor_slices((images, labels))
dataset_tf = dataset_tf.map(preprocess)

# Split dataset (60% train, 20% val, 20% test)
data_size = len(images)
train_size = int(0.6 * data_size)
val_size = int(0.2 * data_size)

dataset_tf = dataset_tf.shuffle(data_size)
train_dataset = dataset_tf.take(train_size).batch(batch_size)
val_dataset = dataset_tf.skip(train_size).take(val_size).batch(batch_size)
test_dataset = dataset_tf.skip(train_size + val_size).batch(batch_size)

# Define a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=num_epochs, callbacks=[early_stopping])

print("Training complete.")


100%|███████████████████████████████████████████████████████████████████████████████| 206M/206M [02:07<00:00, 1.61MB/s]
C:\Users\sukanya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 162s 78ms/step - accuracy: 0.3756 - loss: 1.6528 - val_accuracy: 0.6216 - val_loss: 0.9840
Epoch 2/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 95s 50ms/step - accuracy: 0.6450 - loss: 0.9391 - val_accuracy: 0.7058 - val_loss: 0.8014
Epoch 3/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 135s 42ms/step - accuracy: 0.7117 - loss: 0.7863 - val_accuracy: 0.7532 - val_loss: 0.7036
Epoch 4/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 94s 54ms/step - accuracy: 0.7457 - loss: 0.6990 - val_accuracy: 0.7732 - val_loss: 0.6186
Epoch 5/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 138s 74ms/step - accuracy: 0.7735 - loss: 0.6250 - val_accuracy: 0.7785 - val_loss: 0.5973
Epoch 6/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 42s 23ms/step - accuracy: 0.8397 - loss: 0.4502 - val_accuracy: 0.8538 - val_loss: 0.4088
Epoch 10/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 40s 22ms/step - accuracy: 0.8462 - loss: 0.4307 - val_accuracy: 0.8600 - val_loss: 0.3806
Epoch 11/50
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 40s 23ms/step - accuracy: 0.8

In [4]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(test_dataset, batch_size=batch_size)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

563/563 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9814 - loss: 0.0630
Test Accuracy: 98.09%
